# **演習Ⅰ 課題2**

中間層を2層に変更してコードを実行してください。

各層のノード数は128, 64に設定してください。

※上部にある「ドライブにコピー」で自分のドライブにコピーしてから編集・実行してください。

In [ ]:
# ライブラリのインポート
import os
import time
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

# Google DriveをマウントしてDrive内のファイルにアクセスできるようにする
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 各種設定
BATCH_SIZE = 128 # ミニバッチのサイズ
MAX_EPOCH = 10   # エポック数（訓練データ全体を何回繰り返して学習させるか）

# シード値の固定
# シードを固定すると毎回同じ結果が得られる（再現性の確保）
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

In [ ]:
# データの前処理の方法を定義
transform = transforms.Compose([
	transforms.ToTensor() # データをテンソルに変換
])

In [ ]:
# 訓練データ、検証データ、テストデータの準備
# 手書き数字画像データセットMNISTを使用

# 訓練データの取得
train_dataset = datasets.MNIST(
	"./data",            # データの保存先
	train=True,          # 訓練データとして取得
	download=True,       # データが存在しない場合はダウンロード
	transform=transform, # データに前処理を適用
)

# 訓練データの一部を検証データとして分割
train_dataset, valid_dataset = torch.utils.data.random_split(
	  train_dataset,
	  [48000, 12000] # 48000枚を訓練データ、12000枚を検証データに分割
)

# テストデータの取得
test_dataset = datasets.MNIST(
	"./data",            # データの保存先
	train=False,         # テストデータとして取得
	download=True,       # データが存在しない場合はダウンロード
	transform=transform, # データに前処理を適用
)

In [ ]:
# データローダーの作成
# データセットからバッチを作成してモデルに供給する役割

# 訓練データ用のデータローダー
train_loader = torch.utils.data.DataLoader(
	train_dataset,
	batch_size=BATCH_SIZE, # バッチサイズごとにデータを供給
	shuffle=True,          # 学習時はデータをシャッフル
)
# 検証データ用のデータローダー
valid_loader = torch.utils.data.DataLoader(
	valid_dataset,
	batch_size=BATCH_SIZE, # バッチサイズごとにデータを供給
	shuffle=False,         # 検証時はシャッフルしない
)
# テストデータ用のデータローダー
test_loader = torch.utils.data.DataLoader(
	test_dataset,
	batch_size=BATCH_SIZE, # バッチサイズごとにデータを供給
	shuffle=False,         # テスト時もシャッフルしない
)

In [ ]:
# ニューラルネットワークの定義
class MLP(nn.Module):
	def __init__(self):
		'''
		ニューラルネットワークの構造を定義
		今回は「多層パーセプトロン（MLP）」というモデルを使用
		'''
		super(MLP, self).__init__()

		######## ↓ 課題2 編集部分① ########

		# 入力層から中間層への変換を定義
		# 入力層のノード数：784個（28x28ピクセルの画像を一列のベクトルに変換）
		# 中間層のノード数：16個
		self.fc1 = nn.Linear(28 * 28, 16)

		# 中間層から出力層への変換を定義
		# 出力層のノード数：10個（数字0〜9を予測）
		self.fc_output = nn.Linear(16, 10)

		######## ↑ 課題2 編集部分① ########

	def forward(self, x):
		'''
		入力から出力までの流れを定義
		'''
		x = x.view(-1, 28 * 28) # 28x28ピクセルの画像を一列のベクトルに変換

		######## ↓ 課題2 編集部分② ########

		x = self.fc1(x)         # 中間層の計算
		x = F.relu(x)           # 活性化関数ReLUを適用
		x = self.fc_output(x)   # 出力層の計算

		######## ↑ 課題2 編集部分② ########

		return x

In [ ]:
# 定義したニューラルネットワークのインスタンス化
model = MLP()

# 損失関数の定義
loss_function = nn.CrossEntropyLoss()

# 最適化手法と学習率の定義
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [ ]:
# 学習を通しての損失を記録するためのリスト（学習曲線のプロット用）
train_losses = []
valid_losses = []

# 学習にかかった時間の計測開始
start_time = time.time()

print("\n【学習開始】\n")
print("train_loss: 学習データにおける損失")
print("valid_loss: 検証データにおける損失\n")

# エポックごとの訓練損失と検証損失を出力
print("epoch\ttrain_loss\tvalid_loss")

for epoch in range(MAX_EPOCH):
	model.train()        # モデルを訓練モードに設定
	train_loss_list = [] # 訓練損失を保存するリスト

	# 訓練データでモデルを学習
	for x, label in train_loader:
		optimizer.zero_grad()               # 勾配をリセット
		output = model(x)                   # モデルの出力を計算
		loss = loss_function(output, label) # 損失を計算
		loss.backward()                     # 勾配を計算
		optimizer.step()                    # 勾配に基づいてパラメータを更新
		train_loss_list.append(loss.item()) # バッチごとの損失を記録

	train_loss_mean = np.mean(train_loss_list) # エポックごとの平均訓練損失
	train_losses.append(train_loss_mean)       # 訓練損失をリストに追加

	model.eval()         # モデルを評価モードに設定
	valid_loss_list = [] # 検証損失を保存するリスト

	# 検証データで損失を計算
	for x, label in valid_loader:
		output = model(x)                      # モデルの出力を計算
		loss = loss_function(output, label)    # 損失を計算
		valid_loss_list.append(loss.item())    # バッチごとの損失を記録

	valid_loss_mean = np.mean(valid_loss_list) # エポックごとの平均検証損失
	valid_losses.append(valid_loss_mean)       # 検証損失をリストに追加

	print("{}\t{:.8}\t{:.8}".format(epoch, train_loss_mean, valid_loss_mean))

# 学習にかかった時間の計測終了
end_time = time.time()
print("\n学習にかかった時間 : {} [sec]".format(end_time - start_time))

print("\n【学習終了】\n")

In [ ]:
# 学習曲線のプロット
plt.plot(range(MAX_EPOCH), train_losses, label='Training Loss')
plt.plot(range(MAX_EPOCH), valid_losses, label='Validation Loss', color="green")
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Learning Curve')
plt.legend()
plt.show()

In [ ]:
# テストデータを使ってモデルを評価
model.eval() # モデルを評価モードに設定
test_loss = 0
test_correct = 0
test_total = 0

# テストデータでの損失と正解率を計算
for x, label in test_loader:
	output = model(x)                            # モデルの出力を計算
	loss = loss_function(output, label)          # 損失を計算
	_, pred = torch.max(output.data, dim=1)      # 出力の最大値を持つクラスを予測
	test_correct += (pred == label).sum().item() # 正解の数をカウント
	test_total += label.size()[0]                # テストデータの総数をカウント

# テストデータにおける正解率を計算して表示
test_accuracy = test_correct / test_total
print("テストデータにおける正解率 : {:.4f}".format(test_accuracy))

In [ ]:
# モデルの保存
model_dir = "/content/drive/MyDrive/jts2024_2/model/"
os.makedirs(model_dir, exist_ok=True)
save_path = model_dir + "mlp_2.pt"
torch.save(model.state_dict(), save_path)